## Chemical energy simulation
The simulation of molecules has been a sore spot for computational chemists and computer scientists ever since the derivation of the Schrödinger equation. The reason? There are simply too many different variables at play. As computational complexity increases, these problems have not become easier.

## Real World Motivations
There aren't many currently available ground state estimators, and many of them are constrained to using already available basis sets (PyScf) or by the computational complexity of the simulation (Qiskit). Being able to accurately calculate the ground state energy of molecules in a short amount of time has applications in drug discovery, computational chemistry, and mateerial science. For many of these fields, chemical simulations is one of the biggest wasters of time, so speeding simulations up would mean saving time-related costs and computational resources.

In [167]:
import bqphy.BQPhy_Optimiser as BQPOptimizer


from fractions import Fraction
import numpy as np
from scipy.constants import e, epsilon_0
import random

## Fitness Function and Dependent Variables
In chemical simulation, the following Hamiltonian, also called the Coulomb Hamiltonian, is central formula.
#### $$\hat{H} =  -\sum_{i} \frac{\hbar^{2}}{2M_{i}} \nabla_{R_{i}}^{2} - \sum_{i} \frac{\hbar^{2}}{2m_{e}}\nabla_{r_{i}}^{2} -\sum_{i} \sum_{j} \frac{Z_{i}e^{2}}{4\pi\epsilon_{0}|R_{i} - r_{j}|} + \sum_{i} \sum_{j > i} \frac{e^{2}}{4\pi\epsilon_{0}|r_{i} - r_{j}|} + \sum{i}\sum_{j > i} \frac{Z_{i}Z_{j}e^{2}}{4\pi\epsilon_{0} |R_{i} - R_{j}|} $$
It defines the kinetic energy of the nuclei, kinetic energy of the electrons, the potential energy between the electrons and nuclei, the potential energy between electrons, and the potential energy of nuclei repulsion. The kinetic energy is impossible to solve analytical right now, since it requires solving for the Schrödinger equation for more than 2 bodies. For this notebook, we're focusing on just the potential electronic energy of the system, denoted by the following, reduced Hamiltonian.
#### $$\hat{H} = -\sum_{i} \sum_{j} \frac{Z_{i}e^{2}}{4\pi\epsilon_{0}|R_{i} - r_{j}|} + \sum_{i} \sum_{j > i} \frac{e^{2}}{4\pi\epsilon_{0}|r_{i} - r_{j}|}$$
As you can see, we're ignoring three terms within the typical Coulombic molecular Hamiltonian. Namely, the kinetic energy terms for the nucleus and the electrons, as well as the potential energy term for the nuclei-nuclei repulsions.

For this notebook, we will be looking exclusively at 1:1 ratios of proton/neutrons and electrons. This simplifies up the molecular Hamiltonian to an easily reducible form. 

## Decision Variables
The decision variables will be in the format of $[x_{1}, y_{1}, x_{2}, y_{2}, \dots, x_{n}, y_{n}]$ where $n$ is the atomic number of the element we're looking for. Every single entry represents the position of an electron in 2-dimensional space. This then provides the fitness function, which optimizes based on length between entities. 

## Constraints
There is only one constraint in this model, the distance between the electrons and the nuclei. We will be using the Bohr radius, which is the distance between the electron and the proton in a hydrogen atom, which is around 52.9 picometers.
$$\sum_{i} \sum_{j} |R_{i} - r_{j}| \geq 52.9$$
This constraint is encapsulated within the lower bounds of the decision variables.

In [15]:
BOHR = 52.9

## Choice of Element
We're using boron as our element of choice here. This is because hydrogen, helium, lithium, and beryllium all have trivial solutions to their positioning. The ground state electronic energy of boron has also been proven within the literature. 

Schaefer, Henry F., and Frank E. Harris. “Electronic structure of atomic boron.” Physical Review, vol. 167, no. 1, 5 Mar. 1968, pp. 67–73, https://doi.org/10.1103/physrev.167.67. 

In [623]:
element = 5

In [624]:
def atomic_fitness(x):
  fitness = np.zeros(x.shape[0], dtype=np.float64)
  for i in range(x.shape[0]):
    pos_list = x[i]
    electron_pos = np.reshape(pos_list, (-1, 2))
    for j in range(len(electron_pos)):
      for k in range(len(electron_pos)):
        if (k > j):
          fitness[i] += np.square(e) / (4 * np.pi * epsilon_0 * 
                                    (np.sqrt(np.square(electron_pos[j][0] - electron_pos[k][0]) + 
                                np.square(electron_pos[j][1] - electron_pos[k][1])) * (1 / np.pow(10, 12))))
        else:
          continue
        fitness[i] -= (element * np.square(e)) / (4 * np.pi * epsilon_0 * np.sqrt(np.square(electron_pos[j][0]) + np.square(electron_pos[j][1])) * (1 / np.pow(10, 12)))
  return fitness

In [625]:
def bounds_construction(element : int):
  lower_bounds = []
  upper_bounds = []
  for i in range(element):
    if (i // 2 == 0):
      lower_bounds.append(BOHR + random.randint(0, int(BOHR / 4)))
      lower_bounds.append(BOHR + random.randint(0, int(BOHR / 4)))
      upper_bounds.append(2 * BOHR - random.randint(0, int(BOHR / 4)))
      upper_bounds.append(2 * BOHR + random.randint(0, int(BOHR / 4)))
    elif (i // 10 == 0):
      lower_bounds.append(2 * BOHR - random.randint(0, int(BOHR / 4)))
      lower_bounds.append(2 * BOHR - random.randint(0, int(BOHR / 4)))
      upper_bounds.append(4 * BOHR - random.randint(0, int(BOHR / 4)))
      upper_bounds.append(4 * BOHR - random.randint(0, int(BOHR / 4)))
  return lower_bounds, upper_bounds

In [626]:
config = {
  "numPopulation": 100,
  "maxGeneration": 200,
  "deltaTheta": .05,
  "designVariables": element * 2,
  "typeOfOptimisation":"CONTINUOUS"
}
config["lowerBounds"], config["upperBounds"] = bounds_construction(element)

In [627]:
optimizer = BQPOptimizer.BQPhy_OPTIMISER()
optimizer.initialize(config)
optimizer.model(atomic_fitness)

In [628]:
optimizer.runOptimization()

In [629]:
best_solution, best_fitness = optimizer.getBestDesign()

In [630]:
def display_design(solution):
  electrons = np.reshape(solution, (-1, 2))
  for i in range(len(electrons)):
    print(f"Electron {i + 1} x: {electrons[i][0]} y: {electrons[i][1]}")

In [631]:
display_design(best_solution)

Electron 1 x: 81.33669642175937 y: 54.9
Electron 2 x: 57.9 y: 54.9
Electron 3 x: 92.8 y: 131.37465781643397
Electron 4 x: 208.5837033646143 y: 93.8
Electron 5 x: 211.56613717860685 y: 205.5916227969787


In [632]:
print(f"Best fitness: {best_fitness[-1] * (10 ** 19)} eV")

Best fitness: -831.5526520682264 eV


The ground state electronic energy of boron, according to the literature, is around -24.6392 hartrees, or -670.46673195 eV. The difference in value could be attributed to the kinetic energy of the electrons and electric potential from the nuceli to nuceli interaction being omitted. Another source of difference is the correlation energy, which is the loss of energy in calculation due to instantenous electron-electron interactions that may happen in an atom due to the movement of electrons.

## Areas of Further Inquiry
Increasing the accuracy of the estimation would be the top priority. In our tests, it's shown that the result has quite a bit of variation, which is most likely due to the convergence of the populations. Another area to focus on would be adding nuclei-nuceli interactions, as it also accounts for a good portion of the total potential energy. 